# 整理 ESM vs. DSM (1D Gaussian)

File: 20251029_白板 DSM vs. ESM_diffusion model_test03  

In [2]:
import math, os, random, numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
import matplotlib.pyplot as plt

# ----- Repro -----
SEED = 1234
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)

DEVICE = torch.device("cpu")

# ----- Data spec -----
MU = 3.0                 # data mean
SIG_DATA = 1.0           # sigma_data
# noise levels to try for DSM (will train one per sigma)
DSM_NOISES = [0.1, 0.3, 0.7, 1.0]

N_TRAIN = 50_000
N_VAL   = 10_000
N_TEST  = 10_000

# ----- Train config -----
BATCH = 256
LR    = 1e-3
STEPS = 4_000
PRINT_EVERY = 200
EMA_DECAY = 0.999


## Model

In [ ]:
class ScoreMLP(nn.Module):
    def __init__(self, hidden=64):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(1, hidden),
            nn.SiLU(),
            nn.Linear(hidden, hidden),
            nn.SiLU(),
            nn.Linear(hidden, 1),
        )
        # small init on final to stabilize early training
        nn.init.zeros_(self.net[-1].weight); nn.init.zeros_(self.net[-1].bias)

    def forward(self, x):
        return self.net(x)
    
